In [2]:
"""
Adaptive_self_rag
금융상품(예: 정기예금, 입출금자유예금) 관련 질의에 대해:
1. 질문 라우팅 → (금융상품 관련이면) 문서 검색 (병렬 서브 그래프) → 문서 평가 → (조건부) 질문 재작성 → 답변 생성
   / (금융상품과 무관하면) LLM fallback을 통해 바로 답변 생성
그리고 생성된 답변의 품질(환각, 관련성) 평가 후 필요시 재생성 또는 재작성하는 Adaptive Self-RAG 체인.
"""


#############################
# 1. 기본 환경 및 라이브러리
#############################

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

# 기타 유틸
import json
import uuid
from pprint import pprint
from textwrap import dedent
from operator import add


# LangChain, Chroma, LLM 관련
from typing import List, Literal, Sequence, TypedDict, Annotated, Tuple
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver

# Grader 평가지표용
from pydantic import BaseModel, Field

# 그래프 관련
from langgraph.graph import StateGraph, START, END

# Gradio 관련
import gradio as gr


In [3]:
from langchain_community.tools import TavilySearchResults
from langchain_core.runnables import RunnableLambda

In [1]:
#############################
# 4. LLM 초기화 & 도구 바인딩
#############################

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)

#############################
# 5. LLM 체인 (Retrieval Grader / Answer Generator / Hallucination / Answer Graders / Question Re-writer)
#############################
print("\n===================================================================\n ")
print("LLM 체인\n")
print("# (1) Retrieval Grader\n")

# (1) Retrieval Grader (검색평가)
class BinaryGradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

structured_llm_BinaryGradeDocuments = llm.with_structured_output(BinaryGradeDocuments)

system_prompt = """You are an expert in evaluating the relevance of search results to user queries.

[Evaluation criteria]
1. 키워드 관련성: 문서가 질문의 주요 단어나 유사어를 포함하는지 확인
2. 의미적 관련성: 문서의 전반적인 주제가 질문의 의도와 일치하는지 평가
3. 부분 관련성: 질문의 일부를 다루거나 맥락 정보를 제공하는 문서도 고려
4. 답변 가능성: 직접적인 답이 아니더라도 답변 형성에 도움될 정보 포함 여부 평가

[Scoring]
- Rate 'yes' if relevant, 'no' if not
- Default to 'no' when uncertain

[Key points]
- Consider the full context of the query, not just word matching
- Rate as relevant if useful information is present, even if not a complete answer

Your evaluation is crucial for improving information retrieval systems. Provide balanced assessments.
"""
# 채점 프롬프트 템플릿
grade_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "[Retrieved document]\n{document}\n\n[User question]\n{question}")
])

retrieval_grader_binary = grade_prompt | structured_llm_BinaryGradeDocuments

question = "어떤 예금 상품이 있는지 설명해주세요."
print(f'\nquestion : {question}\n')
retrieved_docs = fixed_deposit_db.similarity_search(question, k=2)
print(f"검색된 문서 수: {len(retrieved_docs)}")
print("===============================================================================")
print()

relevant_docs = []
for doc in retrieved_docs:
    print("문서:\n", doc.page_content)
    print("---------------------------------------------------------------------------")

    relevance = retrieval_grader_binary.invoke({"question": question, "document": doc.page_content})
    print(f"문서 관련성: {relevance}")

    if relevance.binary_score == 'yes':
        relevant_docs.append(doc)
    
    print("===========================================================================")

print("\n# (2) Answer Generator (일반 RAG) \n")

# (2) Answer Generator (일반 RAG)
def generator_rag_answer(question, docs):

    template = """
    [Your task]
    You are a financial product expert and consultant who always responds in Korean.
    Your task is to analyze the user query and the given financial product data to recommend the most suitable financial product.
    
    [Instructions]
    1. 질문과 관련된 정보를 문맥에서 신중하게 확인합니다.
    2. 답변에 질문과 직접 관련된 정보만 사용합니다.
    3. 문맥에 명시되지 않은 내용에 대해 추측하지 않습니다.
    4. 불필요한 정보를 피하고, 답변을 간결하고 명확하게 작성합니다.
    5. 문맥에서 정확한 답변을 생성할 수 없다면 최대한 필요한 답변을 생성한 뒤 마지막에 "더 구체적인 정보를 알려주시면 더욱 명쾌한 답변을 할 수 있습니다."라고 덧붙여 답변합니다.
    6. 적절한 경우 문맥에서 직접 인용하며, 따옴표를 사용합니다.

    [Context]
    {context}

    [Question]
    {question}

    [Answer]
    """

    prompt = ChatPromptTemplate.from_template(template)
    local_llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    def format_docs(docs):
        return "\n\n".join([d.page_content for d in docs])
    
    rag_chain = prompt | local_llm | StrOutputParser()
    generation = rag_chain.invoke({"context": format_docs(docs), "question": question})
    return generation

generation = generator_rag_answer(question, docs=relevant_docs)
print("Generated Answer (일반 RAG):")
print(generation)

# (3) Hallucination Grader
print("\n# (3) Hallucination Grader\n")

class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""
    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

structured_llm_HradeHallucinations = llm.with_structured_output(GradeHallucinations)

# 환각 평가를 위한 시스템 프롬프트 정의
halluci_system_prompt = """
You are an expert evaluator assessing whether an LLM-generated answer is grounded in and supported by a given set of facts.

[Your task]
    - Review the LLM-generated answer.
    - Determine if the answer is fully supported by the given facts.

[Evaluation criteria]
    - 답변에 주어진 사실이나 명확히 추론할 수 있는 정보 외의 내용이 없어야 합니다.
    - 답변의 모든 핵심 내용이 주어진 사실에서 비롯되어야 합니다.
    - 사실적 정확성에 집중하고, 글쓰기 스타일이나 완전성은 평가하지 않습니다.

[Scoring]
    - 'yes': The answer is factually grounded and fully supported.
    - 'no': The answer includes information or claims not based on the given facts.

Your evaluation is crucial in ensuring the reliability and factual accuracy of AI-generated responses. Be thorough and critical in your assessment.
"""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", halluci_system_prompt),
        ("human", "[Set of facts]\n{documents}\n\n[LLM generation]\n{generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_HradeHallucinations
hallucination = hallucination_grader.invoke({"documents": relevant_docs, "generation": generation})
print(f"환각 평가: {hallucination}")

print("\n# (4) Answer Grader\n")
# (4) Answer Grader 
class BinaryGradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""
    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

structured_llm_BinaryGradeAnswer = llm.with_structured_output(BinaryGradeAnswer)
grade_system_prompt = """
You are an expert evaluator tasked with assessing whether an LLM-generated answer effectively addresses and resolves a user's question.

[Your task]
    - Carefully analyze the user's question to understand its core intent and requirements.
    - Determine if the LLM-generated answer sufficiently resolves the question.

[Evaluation criteria]
    - 관련성: 답변이 질문과 직접적으로 관련되어야 합니다.
    - 완전성: 질문의 모든 측면이 다뤄져야 합니다.
    - 정확성: 제공된 정보가 정확하고 최신이어야 합니다.
    - 명확성: 답변이 명확하고 이해하기 쉬워야 합니다.
    - 구체성: 질문의 요구 사항에 맞는 상세한 답변이어야 합니다.

[Scoring]
    - 'yes': The answer effectively resolves the question.
    - 'no': The answer fails to sufficiently resolve the question or lacks crucial elements.

Your evaluation plays a critical role in ensuring the quality and effectiveness of AI-generated responses. Strive for balanced and thoughtful assessments.
"""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", grade_system_prompt),
        ("human", "[User question]\n{question}\n\n[LLM generation]\n{generation}"),
    ]
)

answer_grader_binary = answer_prompt | structured_llm_BinaryGradeAnswer
print("Question:", question)
print("Generation:", generation)
answer_score = answer_grader_binary.invoke({"question": question, "generation": generation})
print(f"답변 평가: {answer_score}")


print("\n# (5) Question Re-writer\n")
# (5) Question Re-writer
def rewrite_question(question: str) -> str:
    """
    입력 질문을 벡터 검색에 최적화된 형태로 재작성한다.
    """
    local_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    system_prompt = """
    You are an expert question re-writer. Your task is to convert input questions into optimized versions 
    for vectorstore retrieval. Analyze the input carefully and focus on capturing the underlying semantic 
    intent and meaning. Your goal is to create a question that will lead to more effective and relevant 
    document retrieval.

    [Guidelines]
        1. Identify and emphasize core concepts and key subjects.
        2. Expand abbreviations or ambiguous terms.
        3. Include synonyms or related terms that might appear in relevant documents.
        4. Maintain the original intent and scope.
        5. For complex questions, break them down into simpler, focused sub-questions.
    """
    re_write_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "[Initial question]\n{question}\n\n[Improved question]\n")
    ])
    question_rewriter = re_write_prompt | local_llm | StrOutputParser()
    rewritten_question = question_rewriter.invoke({"question": question})
    return rewritten_question

print("\n# (6) Generation Evaluation & Decision Nodes\n")
# (6) Generation Evaluation & Decision Nodes
def grade_generation_self(state: "SelfRagOverallState") -> str:
    print("--- 답변 평가 (생성) ---")
    print(f"--- 생성된 답변: {state['generation']} ---")
    if state['num_generations'] > 2:
        print("--- 생성 횟수 초과, 종료 -> end ---")
        return "end"
    # 평가를 위한 문서 텍스트 구성
    print("--- 답변 할루시네이션 평가 ---")
    docs_text = "\n\n".join([d.page_content for d in state['documents']])
    hallucination_grade = hallucination_grader.invoke({
        "documents": docs_text,
        "generation": state['generation']
    })
    if hallucination_grade.binary_score == "yes":
        relevance_grade = retrieval_grader_binary.invoke({
            "question": state['question'],
            "generation": state['generation']
        })
        print("--- 답변-질문 관련성 평가 ---")
        if relevance_grade.binary_score == "yes":
            print("--- 생성된 답변이 질문을 잘 해결함 ---")
            return "useful"
        else:
            print("--- 답변 관련성이 부족 -> transform_query ---")
            return "not useful"
    else:
        print("--- 생성된 답변의 근거가 부족 -> generate 재시도 ---")
        return "not supported"
    
def decide_to_generate_self(state: "SelfRagOverallState") -> str:
    print("--- 평가된 문서 분석 ---")
    if state['num_generations'] > 2:
        print("--- 생성 횟수 초과, 생성 결정 ---")
        return "generate"
    # 여기서는 필터링된 문서가 존재하는지 확인
    if not state['filtered_documents']:
        print("--- 관련 문서 없음 -> transform_query ---")
        return "transform_query"
    else:
        print("--- 관련 문서 존재 -> generate ---")
        return "generate"


# (7) RoutingDecision 
class RoutingDecision(BaseModel):
    """Determines whether a user question should be routed to document search or LLM fallback."""
    route: Literal["search_data","llm_fallback"] = Field(
        description="Classify the question as 'search_data' (financial) or 'llm_fallback' (general)"
        )

NameError: name 'ChatOpenAI' is not defined